In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Spark SQL Query DataFrame').getOrCreate()

24/05/06 10:13:01 WARN Utils: Your hostname, aliyu-XPS-15-9550 resolves to a loopback address: 127.0.1.1; using 192.168.199.55 instead (on interface wlp2s0)
24/05/06 10:13:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/06 10:13:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/06 10:13:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data_path = './Data/'

In [4]:
json_df2_path = data_path + 'utilization.json'
df = spark.read.format('json').load(json_df2_path)

In [5]:
df.show(10)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
+---------------+-------------------+-

In [6]:
df.createOrReplaceTempView('utilization')

In [7]:
df_count = spark.sql('SELECT count(*) FROM utilization')
df_count.show()

+--------+
|count(1)|
+--------+
|  500000|
+--------+



In [8]:
df_sql = spark.sql('SELECT count(*) FROM utilization WHERE session_count > 70')
df_sql.show()

+--------+
|count(1)|
+--------+
|  239659|
+--------+



In [9]:
df_sql = spark.sql('SELECT server_id, count(*) FROM utilization WHERE session_count > 70 GROUP BY server_id')
df_sql.show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      103|    8744|
|      104|    7366|
|      100|     391|
|      105|    1110|
|      101|    9808|
|      102|    8586|
|      112|    7425|
|      113|    9418|
|      110|    2826|
|      107|    5646|
|      111|    3093|
|      108|    8375|
|      109|    3129|
|      116|    1167|
|      114|    2128|
|      115|    5284|
|      120|    2733|
|      118|    7913|
|      117|    3605|
|      126|    6365|
+---------+--------+
only showing top 20 rows



In [10]:
df_sql = spark.sql('SELECT server_id, count(*) FROM utilization WHERE session_count > 70 GROUP BY server_id ORDER BY count(*) DESC')
df_sql.show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      101|    9808|
|      113|    9418|
|      145|    9304|
|      103|    8744|
|      102|    8586|
|      133|    8583|
|      108|    8375|
|      149|    8288|
|      137|    8248|
|      148|    8027|
|      123|    7918|
|      118|    7913|
|      112|    7425|
|      139|    7383|
|      104|    7366|
|      121|    7084|
|      142|    7084|
|      146|    7072|
|      126|    6365|
|      144|    6220|
+---------+--------+
only showing top 20 rows



In [12]:
df_sql = spark.sql('SELECT server_id, avg(session_count),min(session_count), max(session_count) FROM utilization WHERE session_count > 70 GROUP BY server_id ORDER BY count(*) DESC')
df_sql.show()

+---------+------------------+------------------+------------------+
|server_id|avg(session_count)|min(session_count)|max(session_count)|
+---------+------------------+------------------+------------------+
|      101| 87.66557911908646|                71|               105|
|      113| 86.96262476109577|                71|               103|
|      145| 86.97732158211522|                71|               103|
|      103| 85.76372369624886|                71|               101|
|      102| 85.71150710458886|                71|               101|
|      133| 85.46720260981009|                71|               100|
|      108|  85.1219104477612|                71|               100|
|      149|  84.9612693050193|                71|                99|
|      137|  85.0061833171678|                71|                99|
|      148| 84.70350068518749|                71|                99|
|      123| 84.53220510229856|                71|                98|
|      118| 84.65777833944142|    

In [14]:
df_sql = spark.sql('SELECT server_id, min(session_count), round(avg(session_count),2), max(session_count) FROM utilization WHERE session_count > 70 GROUP BY server_id ORDER BY count(*) DESC')
df_sql.show()

+---------+------------------+----------------------------+------------------+
|server_id|min(session_count)|round(avg(session_count), 2)|max(session_count)|
+---------+------------------+----------------------------+------------------+
|      101|                71|                       87.67|               105|
|      113|                71|                       86.96|               103|
|      145|                71|                       86.98|               103|
|      103|                71|                       85.76|               101|
|      102|                71|                       85.71|               101|
|      133|                71|                       85.47|               100|
|      108|                71|                       85.12|               100|
|      149|                71|                       84.96|                99|
|      137|                71|                       85.01|                99|
|      148|                71|                      